## SRJ

In [1]:
# Take 6 excitations from ash_excitation and their parameters
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore")
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=7) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668933
Total excitations are 26
The adapt iteration now is 0
Highest gradient excitation is [2, 3, 6, 7]
Final updated parameters: [1.12299252]
Final cost: -1.4300836708403637
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(1.12)─┤  State
3: ──X─├FermionicDoubleExcitation(1.12)─┤  State
6: ────├FermionicDoubleExcitation(1.12)─┤  State
7: ────╰FermionicDoubleExcitation(1.12)─┤  State
The adapt iteration now is 1
Highest gradient excitation is [0, 3, 5, 6]
Final updated parameters: [1.09716256 1.37775773]
Final cost: -1.5694603835520744
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(1.38)─┤  State
1: ──X──────────────────────────────────├FermionicDoubleExcitation(1.38)─┤  State
2: ──X─╭FermionicDoubleExcitation(1.10)─├FermionicDoubleExcitation(1.38)─┤  State
3: ──X─├FermionicDoubleExc

In [2]:
qubits
active_electrons
Perfect_exc= [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4], [3, 7], [2, 6], [2, 3, 6, 7], [1, 5], [0, 1, 4, 5], [0, 1, 6, 7], [2, 3, 4, 5], [1, 2, 5, 6], [0, 3, 4, 7], [0, 2, 4, 6], [1, 3, 5, 7], [0, 3, 5, 6], [3, 7], [2, 6], [0, 2, 4, 6], [1, 5], [0, 3, 5, 6], [2, 3, 6, 7], [2, 3, 6, 7], [2, 3, 6, 7]]


4

## Calculated Overlap matrix <(G|HF)^dagger | s6>

In [3]:
hf_state = np.array([1,1,1,1,0,0,0,0])


optimizer = qml.GradientDescentOptimizer(stepsize=2.0)

dev = qml.device('lightning.qubit', wires = qubits)
@qml.qnode(dev)
def s6(params, ash_excitation):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
        elif len(ash_excitation[i]) == 2:
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
    return qml.state()

#S6 state is generated


singles, doubles = qml.qchem.excitations(active_electrons, qubits)

op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]

op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
print('Total excitations are', len(operator_pool))



for i in range(len(singles)):
    print('The single excitation going in ', singles[i])
    theta = qml.numpy.array(np.pi/4, requires_grad = True)
    @qml.qnode(dev)
    def ind_sp_e(theta):
        qml.BasisState(hf_state, wires=range(qubits))
        qml.SingleExcitation(theta, wires=singles[i])
        return qml.expval(H)
    
    def cost(x):
        return ind_sp_e(x)
    
    for n in range(300):
        theta, energy  = optimizer.step_and_cost(cost, theta)
    print('Value of theta after optimization', theta)
    print('Valeu of cost', energy)

    @qml.qnode(dev)
    def ind_sp(singles):
        qml.BasisState(hf_state, wires=range(qubits))
        print('The value of theta after optimization', theta)
        qml.SingleExcitation(theta, wires=singles[i])
        return qml.state()
    
    leftstate = ind_sp(singles)
    
    right_op = s6(params, ash_excitation)

    left_op = leftstate.conj().T
    ovlp_matrix_singles = left_op.dot(right_op).real
    print('The value of overlap matrix is ', ovlp_matrix_singles)

    print('\n')

print('\n')
print('Moving to doubles')

for i in range(len(doubles)):
    print('The single excitation going in ', doubles[i])
    theta2 = qml.numpy.array(np.pi/4, requires_grad = True)
    @qml.qnode(dev)
    def ind_dp_e(theta2):
        qml.BasisState(hf_state, wires=range(qubits))
        qml.DoubleExcitation(theta2, wires=doubles[i])
        return qml.expval(H)
    
    def dcost(x):
        return ind_dp_e(x)
    
    for n in range(300):
        theta2, energy  = optimizer.step_and_cost(dcost, theta2)
    print('Value of theta2 after optimization', theta2)
    print('Valeu of cost- DE', energy)


    

    @qml.qnode(dev)
    def ind_dp(theta2):
        print('Value of DE2 theta2', theta2)
        qml.BasisState(hf_state, wires=range(qubits))
        qml.DoubleExcitation(theta2, wires=doubles[i])
        return qml.state()
    
    leftstate = ind_dp(theta2)
    
    left_op_d = leftstate.conj().T
    ovlp_matrix_doubles = left_op_d.dot(right_op).real
    print('The value of overlap matrix is ', ovlp_matrix_doubles)
    print('\n')

Total excitations are 26
The single excitation going in  [0, 4]
Value of theta after optimization -1.3781185926586887e-06
Valeu of cost -1.3325688880669257
The value of theta after optimization -1.3781185926586887e-06
The value of overlap matrix is  0.47363445209136285


The single excitation going in  [0, 6]
Value of theta after optimization 1.354318845823534e-09
Valeu of cost -1.3325688880668944
The value of theta after optimization 1.354318845823534e-09
The value of overlap matrix is  0.47363445209147526


The single excitation going in  [1, 5]
Value of theta after optimization 1.379584660475149e-06
Valeu of cost -1.332568888066926
The value of theta after optimization 1.379584660475149e-06
The value of overlap matrix is  0.47363445209136257


The single excitation going in  [1, 7]
Value of theta after optimization 1.354318845823647e-09
Valeu of cost -1.3325688880668944
The value of theta after optimization 1.354318845823647e-09
The value of overlap matrix is  0.47363445209147526




In [14]:
ash_excitation

[[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1, 5], [2, 6]]

In [ ]:
0.625936597, 0.6285256926711114, 0.6263941499605081, 0.6285198406737982, 

[0, 4]

In [4]:
ash_excitation

[[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5]]

# Calculation of overlap matrix of <(G(op)s6)dagger| s6>

# No optimization

In [7]:
hf_state = np.array([1,1,1,1,0,0,0,0])
singles_results = []
doubles_results = []

optimizer = qml.GradientDescentOptimizer(stepsize=2.0)

dev = qml.device('lightning.qubit', wires = qubits)
@qml.qnode(dev)
def s6(params, ash_excitation):
    print('Ash excitation in s6 function', ash_excitation)
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
        elif len(ash_excitation[i]) == 2:
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
    return qml.state()

#S6 state is generated
s6 = s6(params, ash_excitation)
right_op = s6

singles, doubles = qml.qchem.excitations(active_electrons, qubits)

op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]

op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
print('Total excitations are', len(operator_pool))



for i in range(len(singles)):
    print('The single excitation going in ', singles[i])
    theta = qml.numpy.array(np.pi/4, requires_grad = True)
    @qml.qnode(dev)
    def ind_sp_e(theta):
        #qml.BasisState(hf_state, wires=range(qubits))
        qml.StatePrep(s6, wires=range(qubits))
        qml.SingleExcitation(theta, wires=singles[i])
        return qml.expval(H)
    
    #def cost(x):
    #    return ind_sp_e(x)
    
    #for n in range(300):
    #    theta, energy  = optimizer.step_and_cost(cost, theta)
    #print('Value of theta after optimization', theta)
    #print('Valeu of cost', energy)

    @qml.qnode(dev)
    def ind_sp(singles):
        #qml.BasisState(hf_state, wires=range(qubits))
        qml.StatePrep(s6, wires=range(qubits))
        print('The value of theta after optimization', theta)
        qml.SingleExcitation(theta, wires=singles[i])
        return qml.state()
    
    leftstate = ind_sp(singles)
    
    #right_op = s6(params, ash_excitation)

    left_op = leftstate.conj().T
    ovlp_matrix_singles = left_op.dot(right_op).real
    print('The value of overlap matrix is ', ovlp_matrix_singles)

    print('\n')
    singles_results.append((i, float(theta), float(ovlp_matrix_singles)))

# Find desired single-excitation results
lowest_overlap_single = min(singles_results, key=lambda x: x[2])
highest_theta_single = max(singles_results, key=lambda x: x[1])


print('\n')
print('Moving to doubles')

for i in range(len(doubles)):
    print('The single excitation going in ', doubles[i])
    theta2 = qml.numpy.array(np.pi/4)
    @qml.qnode(dev)
    def ind_dp_e(theta2):
        #qml.BasisState(hf_state, wires=range(qubits))
        qml.StatePrep(s6, wires=range(qubits))
        qml.DoubleExcitation(theta2, wires=doubles[i])
        return qml.expval(H)
    
    def dcost(x):
        return ind_dp_e(x)
    
    #for n in range(300):
    #    theta2, energy  = optimizer.step_and_cost(dcost, theta2)
    #print('Value of theta2 after optimization', theta2)
    #print('Valeu of cost- DE', energy)


    

    @qml.qnode(dev)
    def ind_dp(theta2):
        print('Value of DE2 theta2', theta2)
        #qml.BasisState(hf_state, wires=range(qubits))
        qml.StatePrep(s6, wires=range(qubits))
        qml.DoubleExcitation(theta2, wires=doubles[i])
        return qml.state()
    
    leftstate = ind_dp(theta2)
    
    left_op_d = leftstate.conj().T
    ovlp_matrix_doubles = left_op_d.dot(right_op).real
    print('The value of overlap matrix is ', ovlp_matrix_doubles)
    doubles_results.append((i, float(theta), float(ovlp_matrix_doubles)))
    print('\n')
# Find desired single-excitation results
lowest_overlap_double = min(doubles_results, key=lambda x: x[2])
highest_theta_double = max(doubles_results, key=lambda x: x[1])
print('\n')
print("\nLowest overlap (doubles): index {}, theta {:.6f}, overlap {:.6f}".format(*lowest_overlap_double))
print("Highest theta (doubles): index {}, theta {:.6f}, overlap {:.6f}".format(*highest_theta_double))

Ash excitation in s6 function [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1, 5], [2, 6], [3, 7]]
Total excitations are 26
The single excitation going in  [0, 4]
The value of theta after optimization 0.7853981633974483
The value of overlap matrix is  0.9437101323492478


The single excitation going in  [0, 6]
The value of theta after optimization 0.7853981633974483
The value of overlap matrix is  0.9519632168280078


The single excitation going in  [1, 5]
The value of theta after optimization 0.7853981633974483
The value of overlap matrix is  0.9437101323492477


The single excitation going in  [1, 7]
The value of theta after optimization 0.7853981633974483
The value of overlap matrix is  0.9493310202374626


The single excitation going in  [2, 4]
The value of theta after optimization 0.7853981633974483
The value of overlap matrix is  0.9519632168280079


The single excitation going in  [2, 6]
The value of theta after optimization 0.7853981633974483
The value of overlap ma

# If you don't optimize theta, its picking wrong operator

## Write a code, it must select highest theta + lowest overlap

## Selecting highest theta + lowest overlap

In [2]:
hf_state = np.array([1,1,1,1,0,0,0,0])
singles_results = []
doubles_results = []

optimizer = qml.GradientDescentOptimizer(stepsize=2.0)

dev = qml.device('lightning.qubit', wires = qubits)
@qml.qnode(dev)
def s6(params, ash_excitation):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
        elif len(ash_excitation[i]) == 2:
            qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
    return qml.state()

#S6 state is generated
s6 = s6(params, ash_excitation)
right_op = s6

singles, doubles = qml.qchem.excitations(active_electrons, qubits)

op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
print('Total excitations are', len(operator_pool))



for i in range(len(singles)):
    print('The single excitation going in ', singles[i])
    theta = qml.numpy.array(np.pi/4, requires_grad = True)


    @qml.qnode(dev)
    def ind_sp_e(theta):
        qml.StatePrep(s6, wires=range(qubits))
        qml.SingleExcitation(theta, wires=singles[i])
        return qml.expval(H)
    
    def cost(x):
        return ind_sp_e(x)
    
    for n in range(300):
        theta, energy  = optimizer.step_and_cost(cost, theta)
    print('Value of theta after optimization', theta)
    print('Valeu of cost', energy)

    @qml.qnode(dev)
    def ind_sp(singles):
        #qml.BasisState(hf_state, wires=range(qubits))
        qml.StatePrep(s6, wires=range(qubits))
        print('The value of theta after optimization', theta)
        qml.SingleExcitation(theta, wires=singles[i])
        return qml.state()
    
    leftstate = ind_sp(singles)
    
    #right_op = s6(params, ash_excitation)

    left_op = leftstate.conj().T
    ovlp_matrix_singles = left_op.dot(right_op).real
    print('The value of overlap matrix is ', ovlp_matrix_singles)

    print('\n')
    singles_results.append((i, float(theta), float(ovlp_matrix_singles)))

# Find desired single-excitation results
lowest_overlap_single = min(singles_results, key=lambda x: x[2])
highest_theta_single = max(singles_results, key=lambda x: x[1])

print("\nLowest overlap (singles): index {}, theta {:.6f}, overlap {:.6f}".format(*lowest_overlap_single))
print("Highest theta (singles): index {}, theta {:.6f}, overlap {:.6f}".format(*highest_theta_single))

print('\n')
print('Moving to doubles')

for i in range(len(doubles)):
    print('The single excitation going in ', doubles[i])
    theta2 = qml.numpy.array(np.pi/4, requires_grad = True)
    @qml.qnode(dev)
    def ind_dp_e(theta2):
        qml.StatePrep(s6, wires=range(qubits))
        qml.DoubleExcitation(theta2, wires=doubles[i])
        return qml.expval(H)
    
    def dcost(x):
        return ind_dp_e(x)
    
    for n in range(300):
        theta2, energy  = optimizer.step_and_cost(dcost, theta2)
    print('Value of theta2 after optimization', theta2)
    print('Valeu of cost- DE', energy)


    @qml.qnode(dev)
    def ind_dp(theta2):
        print('Value of DE2 theta2', theta2)
        qml.StatePrep(s6, wires=range(qubits))
        qml.DoubleExcitation(theta2, wires=doubles[i])
        return qml.state()
    
    leftstate = ind_dp(theta2)
    
    left_op_d = leftstate.conj().T
    ovlp_matrix_doubles = left_op_d.dot(right_op).real
    print('The value of overlap matrix is ', ovlp_matrix_doubles)
    doubles_results.append((i, float(theta2), float(ovlp_matrix_doubles)))
    print('\n')
# Find desired double-excitation results
    lowest_overlap_double = min(doubles_results, key=lambda x: x[2])
    highest_theta_double = max(doubles_results, key=lambda x: x[1])



print("\nLowest overlap (doubles): index {}, theta {:.6f}, overlap {:.6f}".format(*lowest_overlap_double))
print("Highest theta (doubles): index {}, theta {:.6f}, overlap {:.6f}".format(*highest_theta_double))

Total excitations are 26
The single excitation going in  [0, 4]
Value of theta after optimization 0.017397185886419475
Valeu of cost -1.8656945376233518
The value of theta after optimization 0.017397185886419475
The value of overlap matrix is  0.9999720234617302


The single excitation going in  [0, 6]
Value of theta after optimization 3.124431650302578e-17
Valeu of cost -1.865676741081744
The value of theta after optimization 3.124431650302578e-17
The value of overlap matrix is  0.9999999999999748


The single excitation going in  [1, 5]
Value of theta after optimization 0.028617317546920054
Valeu of cost -1.8657188583670077
The value of theta after optimization 0.028617317546920054
The value of overlap matrix is  0.9999243011745209


The single excitation going in  [1, 7]
Value of theta after optimization -2.0891856550171676e-17
Valeu of cost -1.865676741081744
The value of theta after optimization -2.0891856550171676e-17
The value of overlap matrix is  0.9999999999999748


The singl

# This is the correct excitation

In [3]:
doubles[17]

[2, 3, 6, 7]